In [1]:
import pandas as pd
import os
from pprint import pprint
from api_key import *
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [3]:
df = pd.read_csv("dados.csv")
df

,Quantidade,Produtos
0,5,Bolo de Cenoura
1,3,Bolo de Chocolate
2,3,Bolo de Ninho com Nutella
3,2,Bolo de Crocante de amendoin
4,3,Bolo de Abacaxi
5,2,Bolo de Milho
6,1,Mini Bolo Churros
7,2,Mini Bolo de Cenoura
8,1,Mini Bolo de Ninho com Nutella
9,3,Mini Bolo Amenduim


In [4]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant that have acess to informations on the datafreme.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

PythonAstREPLTool(locals=df_locals)

In [6]:
from langchain_experimental.tools.python.tool import PythonAstREPLTool


tools = [PythonAstREPLTool(locals={"df": df})]
llm_with_tools = llm.bind_tools(tools)

In [7]:
from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

In [8]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [11]:
list(agent_executor.stream({"input": "Ainda tem quantos bolos de cenoura?"}))





> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df['Bolos de Cenoura'].sum()"}`


KeyError: 'Bolos de Cenoura'
Invoking: `python_repl_ast` with `{'query': "df['Bolo de Cenoura'].sum()"}`
responded: Parece que houve um erro ao tentar acessar a quantidade de bolos de cenoura no dataframe. Vou verificar a coluna correta e retornar a quantidade de bolos de cenoura disponíveis.

KeyError: 'Bolo de Cenoura'
Invoking: `python_repl_ast` with `{'query': "df['Bolos de Cenoura'].sum()"}`
responded: Desculpe, parece que não encontrei a coluna correta para verificar a quantidade de bolos de cenoura. Vou tentar novamente com outro nome de coluna.

KeyError: 'Bolos de Cenoura'
Invoking: `python_repl_ast` with `{'query': "df['Bolo de Cenoura'].sum()"}`
responded: Ainda não consegui encontrar a coluna correta para verificar a quantidade de bolos de cenoura. Vou tentar mais uma vez com outro nome de coluna.

KeyError: 'Bolo de Cenoura'
Invoking: `python_repl_ast` wi

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()